In [120]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
#from sklearn.preprocessing import MinMaxScaler, StandardScaler


from fuel_model import total_fuel_consumption
from segment_model import calculate_block, calculate_segment

## PREPERATON OF SPEED DATASET

In [121]:
path_speed = 'datasets/SPEED/DOT_Traffic_Speeds_NBE_20241001.csv'
df_speed = pd.read_csv(path_speed)
df_speed

,ID,SPEED,TRAVEL_TIME,STATUS,DATA_AS_OF,LINK_ID,LINK_POINTS,ENCODED_POLY_LINE,ENCODED_POLY_LINE_LVLS,OWNER,TRANSCOM_ID,BOROUGH,LINK_NAME
0,263,0.00,0,-101,10/01/2024 06:29:03 AM,4763657,"40.60846,-74.03933 40.6107805,-74.03207 40.611...",{izvFxx{bMoMkl@_AiB_BaBqFkEwDgEkCeDcKiJ,BBBBBBBB,Verrazano-Narrows-Bridge,4763657,Brooklyn,GOW S VNB W 92ND STREET - BKLYN GANTRY LOWER L...
1,264,0.00,0,-101,10/01/2024 06:29:03 AM,4763655,"40.6083804,-74.039301 40.6107105,-74.03202 40....",kizvFrx{bMqMol@cAsByAcBsFkEoCgE}CqD_LeJ,BBBBBBBB,Verrazano-Narrows-Bridge,4763655,Brooklyn,GOW S VNB W 92ND STREET - BKLYN GANTRY UPPER L...
2,411,0.00,0,-101,10/01/2024 06:29:03 AM,4763652,"40.6040405,-74.052321 40.6047,-74.050301 40.60...",gnyvF~i~bMcCsKaPys@wBuJo@yC,BBBBB,Verrazano-Narrows-Bridge,4763652,Brooklyn,VNB E SI GANTRY UPPER LEVEL - BROOKLYN GANTRY ...
3,417,0.00,0,-101,10/01/2024 06:29:03 AM,4763649,"40.60414,-74.052411 40.60479,-74.050351 40.607...",{nyvFpj~bMaC{KcP{s@qBwJw@yC,BBBBB,Verrazano-Narrows-Bridge,4763649,Brooklyn,VNB W BROOKLYN GANTRY UPPER LEVEL - SI GANTRY ...
4,315,14.29,340,0,10/01/2024 06:29:03 AM,4616364,"40.73744001,-73.85188001 40.737015,-73.853730...",_pswFfewaMtApJv@rIh@nDlFbXhH`]bJla@\\|@`FbHz\fBvG,BBBBBBBBBB,NYC_DOT_LIC,4616364,Queens,LIE W 108TH ST - 84TH ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42865900,191,55.30,77,0,01/01/2021 12:03:03 PM,4620331,"40.8465405,-73.932021 40.84611,-73.93075 40.84...",{yhxFbzfbMtA}FfDyT\\\\\\\\|@iGZ_F?_Fe@oSk@i]Ak...,BBBBBBBBBB,PA-GWBridge,4620331,Bronx,CBE W MORRIS AVE - GWB W AMSTERDAM AVE (U/LVL)
42865901,190,52.19,81,0,01/01/2021 12:03:03 PM,4620332,"40.84671,-73.931941 40.8460306,-73.930181 40.8...",}zhxFryfbMfC_J~BqOlAgIZkFC_Hq@sZ[wb@ToV,BBBBBBBBB,PA-GWBridge,4620332,Bronx,CBE W MORRIS AVE - GWB W AMSTERDAM AVE (L/LVL)
42865902,448,9.32,2078,-101,01/01/2021 12:03:03 PM,4620343,"40.77149,-73.99423 40.7719,-73.99401 40.77481,...",ydzwF\\\\\\\\|~rbMqAk@eQyKkUkOyHiFgGgFyPaLgJ{F...,BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB...,NYC_DOT_LIC,4620343,Manhattan,Westside Hwy N 57th St - GWB
42865903,380,49.08,47,0,01/01/2021 12:03:03 PM,4763648,"40.60399,-74.05225 40.6035506,-74.05359 40.603...",}myvFpi~bMvAjG`AnE`AlExDlPr@~CXxDFlFO`DYtBg@xE...,BBBBBBBBBBBBB,NYC_DOT_LIC,4763648,Staten Island,SIE E VNB E FINGERBOARD ROAD - SI GANTRY UPPER...


In [122]:
df_speed_2024 = df_speed[df_speed['DATA_AS_OF'].str.contains('2024')] # Keep rows where 'DATA_AS_OF' contains '2024'

# function for removing missing values
def remove_missing_values (df, df_name="Dataset"):
    missing_values = df.isnull().sum().sum()  # Total number of missing values
        
    if missing_values == 0:
        print(f"{df_name}: No missing values.")
    else:
        print(f"{df_name}: {missing_values} missing values found.")     
        print(f"Missing values per column in {df_name}:\n{df.isnull().sum()}")
        
        df_clean = df.dropna()      # Remove rows with missing values
        print(f"{df_name}: Cleaned dataset, removed rows with missing values.")
        return df_clean
    
    return df

df_speed_clean = remove_missing_values(df_speed_2024, "Traffic Speed Data 2024")

Traffic Speed Data 2024: No missing values.


In [123]:
print(df_speed_clean['BOROUGH'].unique())

['Brooklyn' 'Queens' 'Manhattan' 'Bronx' 'Staten Island']


In [124]:
df_speed_clean_copy = df_speed_clean.copy()                                                     # Avoid tampering with original dataset

df_speed_clean_copy['DATA_AS_OF'] = pd.to_datetime(df_speed_clean_copy['DATA_AS_OF'])           # Convert 'DATA_AS_OF' column to datetime format
df_speed_clean_copy.set_index('DATA_AS_OF', inplace=True)                                       # Set 'DATA_AS_OF' as the index

df_hourly_avg_speed = df_speed_clean_copy.groupby('BOROUGH').resample('H').mean().reset_index() # Resample the data by hour and calculate the mean speed for each borough
df_hourly_avg_speed['Hour'] = df_hourly_avg_speed['DATA_AS_OF'].dt.hour                         # Extract the hour from 'DATA_AS_OF'

df_mean_speed = df_hourly_avg_speed.groupby(['BOROUGH', 'Hour'])['SPEED'].mean().reset_index()  # Group by the borough and hour, calculate average speed across all dates per borough and hour
df_mean_speed.rename(columns={'SPEED': 'AVG SPEED'}, inplace=True)                              # Rename columns for clarity

df_mean_speed


,BOROUGH,Hour,AVG SPEED
0,Bronx,0,39.066408
1,Bronx,1,40.652314
2,Bronx,2,41.506554
3,Bronx,3,42.275486
4,Bronx,4,42.864847
...,...,...,...
115,Staten Island,19,31.870806
116,Staten Island,20,33.866476
117,Staten Island,21,35.068609
118,Staten Island,22,35.393288


In [125]:
df_hourly_avg_speed['Month'] = pd.to_datetime(df_hourly_avg_speed['DATA_AS_OF']).dt.month   # Create a 'Month' 

# Assign seasons function to New York seasons
def assign_seasons(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'  

df_hourly_avg_speed['Season'] = df_hourly_avg_speed['Month'].apply(assign_seasons)   # Apply the function to create a 'Season' column
df_hourly_avg_speed


,BOROUGH,DATA_AS_OF,ID,SPEED,TRAVEL_TIME,STATUS,LINK_ID,TRANSCOM_ID,Hour,Month,Season
0,Bronx,2024-01-01 00:00:00,210.636364,42.992765,115.871212,-15.303030,4.602849e+06,4.602849e+06,0,1,Winter
1,Bronx,2024-01-01 01:00:00,210.636364,40.283523,181.132576,-14.537879,4.602849e+06,4.602849e+06,1,1,Winter
2,Bronx,2024-01-01 02:00:00,210.636364,39.165417,176.636364,-14.537879,4.602849e+06,4.602849e+06,2,1,Winter
3,Bronx,2024-01-01 03:00:00,210.636364,40.810871,134.916667,-16.068182,4.602849e+06,4.602849e+06,3,1,Winter
4,Bronx,2024-01-01 04:00:00,210.636364,42.794735,151.087121,-15.303030,4.602849e+06,4.602849e+06,4,1,Winter
...,...,...,...,...,...,...,...,...,...,...,...
32910,Staten Island,2024-10-01 02:00:00,396.840000,41.543367,67.810000,-27.606667,4.604739e+06,4.604739e+06,2,10,Autumn
32911,Staten Island,2024-10-01 03:00:00,396.840000,43.477900,59.553333,-24.576667,4.604739e+06,4.604739e+06,3,10,Autumn
32912,Staten Island,2024-10-01 04:00:00,396.840000,45.070767,57.596667,-24.240000,4.604739e+06,4.604739e+06,4,10,Autumn
32913,Staten Island,2024-10-01 05:00:00,396.840000,42.678233,61.853333,-25.586667,4.604739e+06,4.604739e+06,5,10,Autumn


In [126]:
seasonal_speed = df_hourly_avg_speed.groupby(['Season', 'Hour'])['SPEED'].mean().unstack()
seasonal_speed

Hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Season,,,,,,,,,,,,,,,,,,,,,
Autumn,31.215625,32.436335,33.315395,34.284927,35.465976,35.435401,30.867001,28.176126,26.304287,26.349509,...,23.948571,22.320087,21.336331,21.438428,22.658334,25.223547,27.573884,29.648036,30.055466,30.633772
Spring,30.930098,31.762357,32.489419,33.334536,34.460258,34.459636,30.555417,27.867530,26.402634,26.320406,...,23.138777,21.312331,20.473336,20.704963,22.654645,25.907429,27.950677,29.282687,29.734043,30.191715
Summer,31.575330,32.666920,33.552263,34.302902,35.757753,36.047039,33.068850,30.591861,28.499463,27.737192,...,24.375007,23.259612,22.539331,22.794803,24.412994,26.915273,28.562503,29.358128,29.800586,30.448576
Winter,33.179240,33.347055,33.700334,34.568948,35.451585,35.370941,31.606355,29.702763,28.604897,29.046959,...,27.203187,25.458156,24.358868,23.945286,25.478932,28.910840,31.175597,32.441041,32.761728,32.817880


In [127]:
# Converting to DataFrame for later Fuel consumption dataframe
speed_df = pd.DataFrame(df_mean_speed) 
pivot_speed_df = speed_df.pivot(index='BOROUGH', columns='Hour', values='AVG SPEED')  # Pivot the table to have columns as Hours, index as Borough, and values as Speeds
pivot_speed_df

Hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
BOROUGH,,,,,,,,,,,,,,,,,,,,,
Bronx,39.066408,40.652314,41.506554,42.275486,42.864847,41.906630,37.202732,34.332337,32.026076,31.104949,...,26.152099,24.700224,24.040555,24.597485,26.878537,30.205903,33.431258,35.824305,36.623494,37.651741
Brooklyn,24.492842,26.714748,27.981259,28.658292,28.691623,26.409135,22.537607,20.975502,19.690766,19.417916,...,18.332524,17.195141,15.968737,15.878412,17.078015,19.239727,20.323587,21.049899,21.629403,22.976908
Manhattan,16.289045,15.577900,15.608534,16.366234,18.986397,23.486297,22.565244,19.994470,18.092892,17.548558,...,15.897006,14.487832,12.997204,13.150159,14.444787,16.701264,17.872534,18.159837,17.699509,16.679459
Queens,41.486279,42.320438,43.561461,45.078357,46.964305,46.342775,41.209348,37.672128,35.511520,36.352465,...,32.049131,29.898403,29.154812,29.595526,31.804754,35.530050,37.997018,39.482661,40.348711,40.780763
Staten Island,36.618500,36.932927,37.217476,37.643164,38.523763,38.291237,34.714973,32.928525,32.257764,32.135275,...,29.537421,27.996010,27.814478,27.268095,28.654942,31.870806,33.866476,35.068609,35.393288,35.941647


## PREPERATON OF VOLUME DATASET

In [128]:
volume_dfs = []

# Define paths to CSV files in a list
csv_paths = [
    'datasets/VOL/31_AVENUE.csv',
    'datasets/VOL/120_AVENUE.csv',
    'datasets/VOL/126_STREET.csv',
    'datasets/VOL/MAIN_STREET.csv',
    'datasets/VOL/SANFORD_AVENUE.csv'
]

# Iterate through each CSV file, standardize 'Roadway Name', and check differences
for path in csv_paths:
    df = pd.read_csv(path)
    df['Roadway Name'] = df['Roadway Name'].str.upper().str.strip()  # Standardize column

    # Find and print unique values in 'Roadway Name' column
    unique_roadway_names = df['Roadway Name'].unique()
    print(f"Dataset: {path}")
    print(f"Unique 'Roadway Name' values: {unique_roadway_names}\n")

    first_value = df['Roadway Name'].unique()[0]
    df['Roadway Name'] = first_value

    # Create month column and assign seasons
    df['Month'] = pd.to_datetime(df['Date']).dt.month  
    df['Season'] = df['Month'].apply(assign_seasons)

    # Check for missing seasons
    seasons_present = df['Season'].unique()
    expected_seasons = {'Winter', 'Spring', 'Summer', 'Autumn'}
    missing_seasons = expected_seasons - set(seasons_present)

    if missing_seasons:
        print(f"Dataset: {path}")
        print(f"Warning: Missing seasons in the data: {missing_seasons}\n")

    df_vol_cleaned = remove_missing_values(df, "Traffic Vol Data")
    volume_dfs.append(df_vol_cleaned)

# Concat and save DataFrames to a file
df_vol_combined = pd.concat(volume_dfs, ignore_index=True)

with open('datasets/df_vol_combined.pkl', 'wb') as f:
    pickle.dump(df_vol_combined, f)

Dataset: datasets/VOL/31_AVENUE.csv
Unique 'Roadway Name' values: ['31 AVENUE' '31ST AVENUE']

Dataset: datasets/VOL/31_AVENUE.csv

Traffic Vol Data: 24 missing values found.
Missing values per column in Traffic Vol Data:
ID               0
SegmentID        0
Roadway Name     0
From             0
To               0
Direction        0
Date             0
12:00-1:00 AM    0
1:00-2:00AM      0
2:00-3:00AM      0
3:00-4:00AM      0
4:00-5:00AM      0
5:00-6:00AM      0
6:00-7:00AM      0
7:00-8:00AM      0
8:00-9:00AM      0
9:00-10:00AM     0
10:00-11:00AM    0
11:00-12:00PM    0
12:00-1:00PM     2
1:00-2:00PM      2
2:00-3:00PM      2
3:00-4:00PM      2
4:00-5:00PM      2
5:00-6:00PM      2
6:00-7:00PM      2
7:00-8:00PM      2
8:00-9:00PM      2
9:00-10:00PM     2
10:00-11:00PM    2
11:00-12:00AM    2
Month            0
Season           0
dtype: int64
Traffic Vol Data: Cleaned dataset, removed rows with missing values.
Dataset: datasets/VOL/120_AVENUE.csv
Unique 'Roadway Name' values: ['

In [129]:
unique_months = df_vol_combined['Month'].unique()
print("Unique Months:", unique_months)
df_vol_combined

Unique Months: [ 1 10  3 11  2  9  5  4]


,ID,SegmentID,Roadway Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,2:00-3:00AM,...,4:00-5:00PM,5:00-6:00PM,6:00-7:00PM,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,Month,Season
0,1929,76510,31 AVENUE,87 STREET,88 STREET,EB,01/25/2012,56,32,26,...,241.0,304.0,236.0,185.0,129.0,105.0,94.0,66.0,1,Winter
1,1930,76510,31 AVENUE,87 STREET,88 STREET,EB,01/26/2012,55,29,21,...,235.0,281.0,262.0,190.0,127.0,124.0,83.0,55.0,1,Winter
2,1931,76510,31 AVENUE,87 STREET,88 STREET,EB,01/27/2012,45,31,20,...,258.0,294.0,290.0,193.0,169.0,136.0,113.0,85.0,1,Winter
3,1932,76510,31 AVENUE,87 STREET,88 STREET,EB,01/28/2012,101,53,52,...,235.0,251.0,222.0,175.0,165.0,151.0,136.0,125.0,1,Winter
4,1933,76510,31 AVENUE,87 STREET,88 STREET,EB,01/29/2012,118,96,84,...,184.0,163.0,195.0,177.0,144.0,117.0,95.0,67.0,1,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,302,91529,SANFORD AVE,147th Street,149th Street,WB,11/18/2020,35,17,12,...,278.0,308.0,234.0,207.0,152.0,125.0,95.0,53.0,11,Autumn
326,302,91529,SANFORD AVE,147th Street,149th Street,WB,11/19/2020,44,17,5,...,282.0,290.0,251.0,241.0,194.0,113.0,93.0,59.0,11,Autumn
327,302,91529,SANFORD AVE,147th Street,149th Street,WB,11/20/2020,33,18,18,...,300.0,305.0,268.0,232.0,212.0,150.0,140.0,82.0,11,Autumn
328,302,91529,SANFORD AVE,147th Street,149th Street,WB,11/21/2020,51,28,20,...,293.0,318.0,235.0,270.0,181.0,127.0,120.0,75.0,11,Autumn


In [130]:
df_melt = df_vol_combined.copy()

# Rename columns representing hours to 0-23
hourly_columns = df_melt.columns[7:-2]                      # Hourly columns start at index 7 and go to the second to last column
new_column_names = list(range(24))                          # Creating new names for the hours (0-23)
rename_dict = dict(zip(hourly_columns, new_column_names))   # Creating a mapping dictionary
df_melt.rename(columns=rename_dict, inplace=True)           # Apply mapping

# Melt the dataset to convert hourly columns (0-23) into rows
df_vol_season_melt = df_melt.melt(id_vars=['Season', 'Roadway Name', 'From', 'To'],
                                  value_vars=new_column_names,
                                  var_name='Hour',
                                  value_name='Volume')

In [131]:
df_vol_season_melt

,Season,Roadway Name,From,To,Hour,Volume
0,Winter,31 AVENUE,87 STREET,88 STREET,0,56.0
1,Winter,31 AVENUE,87 STREET,88 STREET,0,55.0
2,Winter,31 AVENUE,87 STREET,88 STREET,0,45.0
3,Winter,31 AVENUE,87 STREET,88 STREET,0,101.0
4,Winter,31 AVENUE,87 STREET,88 STREET,0,118.0
...,...,...,...,...,...,...
7915,Autumn,SANFORD AVE,147th Street,149th Street,23,53.0
7916,Autumn,SANFORD AVE,147th Street,149th Street,23,59.0
7917,Autumn,SANFORD AVE,147th Street,149th Street,23,82.0
7918,Autumn,SANFORD AVE,147th Street,149th Street,23,75.0


In [139]:
# Investigating road segment length for fuel calculations
unique_segments = df_vol_season_melt[['Roadway Name', 'From', 'To']].drop_duplicates()  # Select relevant columns: 'Roadway Name', 'From', 'To'

# Print all unique "From-To" segments per roadway
for roadway in unique_segments['Roadway Name'].unique():
    roadway_segments = unique_segments[unique_segments['Roadway Name'] == roadway]
    print(f"Roadway: {roadway}")
    print(roadway_segments[['From', 'To']])

Roadway: 31 AVENUE
           From           To
0     87 STREET    88 STREET
10    87 Street    88 Street
50  87th Street  88th Street
Roadway: 120 AVENUE
             From            To
68     191 STREET    192 STREET
78     191 Street    192 Street
114  191th Street  192th Street
Roadway: 126 STREET
          From         To
132  37 AVENUE  38 AVENUE
142  37 Avenue  38 Avenue
178   37th Ave   38th Ave
Roadway: MAIN STREET
          From         To
196  37 AVENUE  38 AVENUE
206  37 Avenue  38 Avenue
242   37th Ave   38th Ave
Roadway: SANFORD AVE
             From            To
260    147 STREET    149 STREET
276    147 Street    149 Street
312  147th Street  149th Street


# FUEL CONSUMPTION DATAFRAME

In [133]:
# Calculate and assign segment length 
df_vol_season_melt['Segment Length'] = df_vol_season_melt.apply(lambda x: calculate_segment(calculate_block(x['From'], x['To'])), axis=1)

# Group data by 'Roadway Name', 'Hour', and 'Segment Length', summarizing 'Volume' directly
grouped_df = df_vol_season_melt.groupby(['Roadway Name', 'Hour', 'Segment Length'], as_index=False).agg({
    'Volume': 'sum'
})

# Map 'Borough' to 'Roadway Name' 
grouped_df['Borough'] = grouped_df['Roadway Name'].replace({
    '31 AVENUE': 'Queens',
    '120 AVENUE': 'Queens',
    'MAIN STREET': 'Queens',
    '126 STREET': 'Manhattan',
    'SANFORD AVE': 'Queens'
})

# Process speed data, ensuring it matches the format of 'grouped_df'
pivot_speed_df_flat = pivot_speed_df.reset_index().melt(id_vars='BOROUGH', var_name='Hour', value_name='Speed')
pivot_speed_df_flat['Hour'] = pivot_speed_df_flat['Hour'].astype(int)  # Match 'Hour' types for merging

# Merge volume and speed data on 'Borough' and 'Hour' columns
df_merged = pd.merge(
    grouped_df, pivot_speed_df_flat, 
    left_on=['Borough', 'Hour'], 
    right_on=['BOROUGH', 'Hour'], 
    how='left'
)

# Drop the redundant 'BOROUGH' column after the merge
df_merged = df_merged.drop(columns=['BOROUGH'])
df_merged

,Roadway Name,Hour,Segment Length,Volume,Borough,Speed
0,120 AVENUE,0,80.4,5278.0,Queens,41.486279
1,120 AVENUE,1,80.4,3495.0,Queens,42.320438
2,120 AVENUE,2,80.4,2367.0,Queens,43.561461
3,120 AVENUE,3,80.4,1830.0,Queens,45.078357
4,120 AVENUE,4,80.4,2211.0,Queens,46.964305
...,...,...,...,...,...,...
115,SANFORD AVE,19,160.8,18649.0,Queens,35.530050
116,SANFORD AVE,20,160.8,15855.0,Queens,37.997018
117,SANFORD AVE,21,160.8,11745.0,Queens,39.482661
118,SANFORD AVE,22,160.8,10363.0,Queens,40.348711


In [134]:
# Calculate fuel consumption for each row in the merged DataFrame
df_merged['Fuel Consumption'] = df_merged.apply(
    lambda row: total_fuel_consumption(row['Volume'], row['Speed'], row['Segment Length']), axis=1
)

# Rearrange the result as a pivot table including 'Segment Length' in the index
result_fuel_consumption = df_merged.pivot_table(
    index=['Roadway Name', 'Segment Length'],
    columns='Hour',
    values='Fuel Consumption',
    aggfunc='sum'
)

with open('datasets/df_merged.pkl', 'wb') as f:
     pickle.dump(df_merged, f)

df_merged

,Roadway Name,Hour,Segment Length,Volume,Borough,Speed,Fuel Consumption
0,120 AVENUE,0,80.4,5278.0,Queens,41.486279,238940
1,120 AVENUE,1,80.4,3495.0,Queens,42.320438,160298
2,120 AVENUE,2,80.4,2367.0,Queens,43.561461,110668
3,120 AVENUE,3,80.4,1830.0,Queens,45.078357,87565
4,120 AVENUE,4,80.4,2211.0,Queens,46.964305,108832
...,...,...,...,...,...,...,...
115,SANFORD AVE,19,160.8,18649.0,Queens,35.530050,1534137
116,SANFORD AVE,20,160.8,15855.0,Queens,37.997018,1357870
117,SANFORD AVE,21,160.8,11745.0,Queens,39.482661,1030194
118,SANFORD AVE,22,160.8,10363.0,Queens,40.348711,921594


In [135]:
# Dataframe that could be used for MOEA, Currently i decided the aggregated version is better for this task, since optimizing lights, and speed limits depending on hours may a bit too much.
# Another optimization that would use this however could be having only lights changed based, on hour of day
result_fuel_consumption

,Hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Roadway Name,Segment Length,,,,,,,,,,,,,,,,,,,,,
120 AVENUE,80.4,238940,160298,110668,87565,108832,154842,331116,662635,693108,545981,...,721597,783590,778392,805847,781689,730996,611362,512465,426114,336228
126 STREET,80.4,104838,60075,47881,42750,61546,125825,176068,277039,377469,434644,...,562422,545826,554460,561331,511195,401882,295755,268402,249055,185146
31 AVENUE,80.4,244146,179149,129650,115797,151312,184153,328186,574886,610910,554278,...,726256,736320,727486,738280,716613,629976,539551,457645,385872,297412
MAIN STREET,80.4,295121,193597,149989,132592,167211,224973,390749,632179,799278,873754,...,965533,932367,897001,917837,910334,882731,790699,659868,578942,422546
SANFORD AVE,160.8,390416,236939,173179,142114,163519,282667,575058,1133500,1579641,1601341,...,1487455,1422673,1397481,1515915,1544741,1534137,1357870,1030194,921594,626432


In [136]:
df_aggregated = df_merged.groupby("Roadway Name").agg({
    "Segment Length": "first",  # Take the first occurrence of Segment Length
    "Volume": "sum",            # Sum the volume across hours
    "Borough": "first",         # Take the first occurrence of Borough
    "Speed": "first",           # Take the first occurrence of Speed
    "Fuel Consumption": "sum"   # Sum fuel consumption across hours
}).reset_index()

with open('datasets/df_aggregated.pkl', 'wb') as f:
     pickle.dump(df_aggregated, f)

df_aggregated

,Roadway Name,Segment Length,Volume,Borough,Speed,Fuel Consumption
0,120 AVENUE,80.4,290273.0,Queens,41.486279,11866272
1,126 STREET,80.4,252902.0,Manhattan,16.289045,7897157
2,31 AVENUE,80.4,280905.0,Queens,41.486279,11505216
3,MAIN STREET,80.4,383042.0,Queens,41.486279,15695532
4,SANFORD AVE,160.8,306435.0,Queens,41.486279,25057920


## OLD CODE

In [137]:
# Unused datasets: 

# Pickle dump datasets to datasets folder
# with open('datasets/df_mean_speed.pkl', 'wb') as f:
#     pickle.dump(df_mean_speed, f)

# with open('datasets/df_hourly_avg_speed', 'wb') as f:
#     pickle.dump(df_hourly_avg_speed, f)

# with open('datasets/df_seasonal_speed.pkl', 'wb') as f:
#     pickle.dump(seasonal_speed, f)


# Save the list of DataFrames to a file
# with open('datasets/df_vol_separate.pkl', 'wb') as f:
#     pickle.dump(volume_dfs, f)


# with open('datasets/df_vol_melt.pkl', 'wb') as f:
#     pickle.dump(df_vol_season_melt, f)

In [138]:
# Code for prepearing for speed and volum merge

# Create new column with segmment length
# df_vol_season_melt['Segment Length'] = df_vol_season_melt.apply(lambda x: calculate_segment(calculate_block(x['From'], x['To'])), axis=1)

# # Now adjust the grouping to preserve 'Segment Length' and volume
# grouped_df = df_vol_season_melt.groupby(['Roadway Name', 'Hour']).agg({
#     'Volume': 'sum',
#     'Segment Length': 'first'  # Assuming segment length is consistent for each 'Roadway Name'
# }).reset_index()

# grouped_df

# # Pivot the table to have hours as columns, and index as 'Roadway Name'
# pivot_vol = grouped_df.pivot_table(index=['Roadway Name', 'Segment Length'], columns='Hour', values='Volume', aggfunc='sum').fillna(0)

# # Ensure all hours from 0 to 23 are present in the pivot table
# all_hours = range(24)
# pivot_vol = pivot_vol.reindex(columns=all_hours, fill_value=0)
# pivot_vol


# # Flatten the volume DataFrame to merge with the speed DataFrame
# pivot_df_flat = pivot_vol.reset_index().melt(id_vars=['Roadway Name', 'Segment Length'], var_name='Hour', value_name='Volume')
# pivot_df_flat

# # Assuming 'pivot_speed_df' is your speed data already pivoted as explained before
# pivot_speed_df_flat = pivot_speed_df.reset_index().melt(id_vars='BOROUGH', var_name='Hour', value_name='Speed')
# pivot_speed_df_flat['Hour'] = pivot_speed_df_flat['Hour'].astype(int)  # Ensure the 'Hour' column types match for merging
